20220518

开始第三版带有晶界模型的测试


In [ ]:
from kmcpy.event_generator import *
from kmcpy.external.pymatgen_structure import Structure
from kmcpy.external.pymatgen_local_env import CutOffDictNN

    
def build_distance_matrix_from_getnninfo_output(cutoffdnn_output={},verbose=False):
    """build a distance matrix from the output of CutOffDictNN.get_nn_info

    nn_info looks like: 
    [{'site': PeriodicSite: Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0), 'weight': 3.7390091507903174, 'site_index': 39, 'wyckoff_sequence': 15, 'local_index': 123, 'label': 'Si1'}, {'site': PeriodicSite: Na+ (-1.2831, -2.6519, 9.2421) [-0.3063, -0.3333, 0.4167], 'image': (-1, -1, 0), 'weight': 3.4778161424304046, 'site_index': 23, 'wyckoff_sequence': 17, 'local_index': 35, 'label': 'Na2'}, {'site': ...]
    
    or say:
    
    nn_info is a list, the elements of list is dictionary, the keys of dictionary are: "site":pymatgen.site, "wyckoff_sequence": ....
    
    Use the site.distance function to build matrix
    

    Args:
        cutoffdnn_output (nn_info, optional): nninfo. Defaults to reference_neighbor_sequences.

    Returns:
        np.2darray: 2d distance matrix, in format of numpy.array. The Column and the Rows are following the input sequence.
    """

    distance_matrix=np.zeros(shape=(len(cutoffdnn_output),len(cutoffdnn_output)))
        

    for sitedictindex1 in range(0,len(cutoffdnn_output)):
        for sitedictindex2 in range(0,len(cutoffdnn_output)):
            """Reason for jimage=[0,0,0]
            
            site.distance is calculated by frac_coord1-frac_coord0 and get the cartesian distance. Note that for the two sites in neighbors,  the frac_coord itself already contains the information of jimage. For exaple:Si4+ (-3.2361, -0.3015, 9.2421) [-0.3712, -0.0379, 0.4167], 'image': (-1, -1, 0),  see that the frac_coord of this Si4+ is not normalized to (0,1)!

            .
            """
            distance_matrix[sitedictindex1][sitedictindex2]=cutoffdnn_output[sitedictindex1]["site"].distance(cutoffdnn_output[sitedictindex2]["site"],jimage=[0,0,0])
        
        
    
    return distance_matrix

li3clo_210=Structure.from_file("grain_boundary_Li3ClO/CONTCAR")
li3clo_210.to("cif","grain_boundary_Li3ClO/210.cif")
li3clo_210.add_oxidation_state_by_guess()

#local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","Li+"):3.0})
#local_env_finder = CutOffDictNN({("Li+","Cl-"):2.8})
np.set_printoptions(precision=2,suppress=True)  
site_to_mark1=[]
site_to_mark2=[]
site_to_mark3=[]
for i in range(0,len(li3clo_210)):
    if "Li+" not in li3clo_210[i].species:
        continue
    
    #print(local_env_finder.get_nn_info(li3clo_210,i))
    print(len(local_env_finder.get_nn_info(li3clo_210,i)))
    """
    
    for #local_env_finder = CutOffDictNN({("Li+","Cl-"):4.0,("Li+","O2-"):4.0})
        if len(local_env_finder.get_nn_info(li3clo_210,i))==5:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"Na+":1.0})

    if len(local_env_finder.get_nn_info(li3clo_210,i))==4:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        li3clo_210.replace(i,{"K+":1.0})
    """
    
    if len(local_env_finder.get_nn_info(li3clo_210,i))!=12:
        print(local_env_finder.get_nn_info(li3clo_210,i))
        print(build_distance_matrix_from_getnninfo_output(local_env_finder.get_nn_info(li3clo_210,i)))
        site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
        li_num=0
        for site in site_contain_li:
            if "Li+" in site:
                li_num+=1
        print("Li num",li_num)

        if len(local_env_finder.get_nn_info(li3clo_210,i))==11:
            site_to_mark1.append(i)
        elif len(local_env_finder.get_nn_info(li3clo_210,i))==8:
            site_contain_li=[neighbor["site"].species for neighbor in local_env_finder.get_nn_info(li3clo_210,i)]
            li_num=0
            for site in site_contain_li:
                if "Li+" in site:
                    li_num+=1
            if li_num==6:
                site_to_mark2.append(i)
            else:
                site_to_mark3.append(i)
        else:
            raise ValueError()

for i in site_to_mark1:
    li3clo_210.replace(i,{"Na+":1.0})
for i in site_to_mark2:
    li3clo_210.replace(i,{"K+":1.0})
for i in site_to_mark3:
    li3clo_210.replace(i,{"Rb+":1.0})
li3clo_210.to("cif","grain_boundary_Li3ClO/marked210.cif")
    
    #print(local_env_finder.get_all_nn_info(li3clo_210))

